In [1]:
import matplotlib.pyplot as plt
import plotly
import plotly.offline as py
import plotly.graph_objs as go
import plotly.figure_factory as ff
py.init_notebook_mode(connected=True)
from blockchain.lib.data.historical import *
from blockchain.lib.model import Seq2Seq
from blockchain.lib.data.dataset import Dataset

In [2]:
pg = PoliniexGetter(period=60*60*2, start_date="2016-07-01")
btc_eth = pg.get(crypto_name=CRYPTO_BTC_ETH, market="poliniex")
btc = pg.get(crypto_name="USDT_BTC", market="poliniex")
eth = pg.get(crypto_name="USDT_ETH", market="poliniex")

In [3]:
config = {
    "log_dir": './seq2seq/seq2seq_2',
    "len_sequence": 10,
    "rnn_layers": [{'num_units': 5}],
    "training_steps": 1000,
    "print_step": 10000000000000,
    "batch_size": 100,
    "ratio_train_val_test": [0.2,0.2],
    "column": ["price"],
    "label": "price",
    "early_stopping": 1000,
    "learning_rate": 1e-5,
    "optimizer": "Adam",
    "y_window": 4,
    "keep_prob": 0.5,
    "output_dim": 1,
    "GO_SYMBOL": -10
}


seq2seq = Seq2Seq(config)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_is_chief': True, '_tf_random_seed': None, '_save_checkpoints_steps': None, '_log_step_count_steps': 100, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f6318d89cc0>, '_environment': 'local', '_save_checkpoints_secs': 600, '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1.0
}
, '_save_summary_steps': 100, '_num_worker_replicas': 0, '_task_type': None, '_master': '', '_model_dir': './seq2seq/seq2seq_2', '_keep_checkpoint_every_n_hours': 10000, '_task_id': 0, '_num_ps_replicas': 0, '_evaluation_master': '', '_session_config': None, '_keep_checkpoint_max': 5}


In [4]:
x, y = Dataset.create_series_dataset(btc, eth, config["column"],
                                             config["label"],
                                             config["len_sequence"],
                                             ratio=config["ratio_train_val_test"],
                                             y_window=config["y_window"])

/media/niki/DATA/private/blockchainanalysis/blockchain/lib/data/dataset/dataset.py:16: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy



In [5]:
y["train"].shape

(3364, 4, 1)

In [6]:
seq2seq.fit(x, y)

Instructions for updating:
Monitors are deprecated. Please use tf.train.SessionRunHook.
Tensor("strided_slice:0", shape=(?, 4, 1), dtype=float32)


NameError: name 'exit' is not defined

In [ ]:
seq2seq.predict(x["train"])